<center><h1><strong>DATA SCRAPY</strong></h1></center>

Universidad Central del Ecuador<br>
Programación para dispositivos móviles<br>
Steven Erraez<br>

# 1. Instalar Pre-Requisitos

In [658]:
!pip install beautifulsoup4 requests pandas

In [659]:
# Load dependencies
import json # modulo para trabajar con archivos JSON
import requests # libreria para hacer peticiones HTTP
import pandas as pd # libreria para procesar datos, proporciona funciones para analíticas
import selenium as sl
from bs4 import BeautifulSoup as bs # Extraer información de paginas web

# 2. Funciones para procesamiento de los datos

In [683]:
# Función que convierte una lista de elementos en un dataframe de pandas
def rowsToDataFrame(rows):
    df = pd.DataFrame(rows[1:74], columns=rows[0])
    return df

In [684]:
# Función para extraer los datos de una tabla HTML
def processTableData(tbl):
    rows = []
    for child in tbl.find('tbody').children: # itera los items internos de la tabla [<tr>]
        row = []
        for td in child: # itera los items internos de la tabla [<td>]
            try:
                item = td.text.replace('\n', '') # recupera el texto del elemento [<td>]
                if item:
                    row.append(item) # Agrega cada elemento de la fila
            except:
                continue
        if len(row) > 0:
            rows.append(row) # agrega todos los campos de una fila de la tabla
    return rows

In [685]:
# Función para exxtrar la data de la URL y procesar el contenido HTML
def processDataHTML(data):
    soup = bs(data['sections'][0]['text'], 'html.parser') # Recupera el item del JSON y lo convierte en contenido HTML
    tbl = soup.find_all('table')[1] # Recupera la tabla del contenido HTML

    #Modificar la tabla para ajustar un formato estandar
    tbl.find_all('tr')[1].decompose()
    tbl.find_all('tr')[0].find_all('th')[3].decompose()
    _chasis_col = soup.new_tag('th', role="column")
    _chasis_col.string = "Chasis"
    _engine_col = soup.new_tag('th', role="column")
    _engine_col.string = "Engine"
    tbl.find_all('tr')[0].insert(6,_chasis_col)
    tbl.find_all('tr')[0].insert(7,_engine_col)
    tbl.find_all('tr')[4].decompose()
    tbl.find_all('tr')[5].decompose()

    tblRows = processTableData(tbl) # Ejecuta la función apra porcesar el contenido de la Tabla
    return tblRows

# 3. Extracción de Información

In [686]:
# Read website
r = requests.get("https://www.wikiwand.com/mcs-api/en.wikipedia.org/v1/page/mobile-sections-remaining/List_of_Formula_One_World_Drivers'_Champions")
table = processDataHTML(json.loads(r.content)) # Formate el contenido en formato JSON\
df = rowsToDataFrame(table) # ejecuta la funcion para recupera los datos de la tabla como dataframe de Pandas

In [687]:
#visualizar tabla
df

,Season,Driver,Age,Chasis,Engine,Tyres,Poles,Wins,Podiums,Fastest laps,Points,% Points,Clinched[17],# of roundsremaining,Margin,% Margin
0,1950,Giuseppe Farina[21],44,Alfa Romeo,Alfa Romeo,P,2,3,3,3,30,83.333 (47.619),Round 7 of 7,0,3,10.000
1,1951,Juan Manuel Fangio[22],40,Alfa Romeo,Alfa Romeo,P,4,3,5,5,31,86.111 (51.389),Round 8 of 8,0,6,19.355
2,1952[lower-alpha 1],Alberto Ascari[24],34,Ferrari,Ferrari,F P,5,6,6,6,36,100.000 (74.306),Round 6 of 8,2,12,33.333
3,1954,Juan Manuel Fangio[22],43,Maserati[lower-alpha 2],Maserati,P,5,6,7,3,42,93.333 (70.547),Round 7 of 9,2,16.857,40.136
4,1955,Juan Manuel Fangio[22],44,Mercedes,Mercedes,C,3,4,5,3,40,88.889 (65.079),Round 6 of 7,1,16.5,41.250
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68,2019,Lewis Hamilton[52],34,Mercedes,Mercedes,P,5,11,17,6,413,75.641,Round 19 of 21,2,87,21.065
69,2020,Lewis Hamilton[52],35,Mercedes,Mercedes,P,10,11,14,6,347,78.507,Round 14 of 17,3,124,35.735
70,2021,Max Verstappen[56],24,Red Bull,Honda,P,10,10,18,6,395.5,69.692,Round 22 of 22,0,8,2.023
71,2022,Max Verstappen[56],25,Red Bull,RBPT,P,7,15,17,5,454,76.174,Round 18 of 22,4,146,32.159


In [688]:
# Inspeccionamos el tipo de dato de cada variable
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 73 entries, 0 to 72
Data columns (total 16 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   Season                73 non-null     object
 1   Driver                73 non-null     object
 2   Age                   73 non-null     object
 3   Chasis                73 non-null     object
 4   Engine                73 non-null     object
 5   Tyres                 73 non-null     object
 6   Poles                 73 non-null     object
 7   Wins                  73 non-null     object
 8   Podiums               73 non-null     object
 9   Fastest laps          73 non-null     object
 10  Points                73 non-null     object
 11  % Points              73 non-null     object
 12  Clinched[17]          73 non-null     object
 13  # of roundsremaining  73 non-null     object
 14  Margin                73 non-null     object
 15  % Margin              73 non-null     obje

# 4. Almacenamiento de los datos

In [689]:
# Guardamos las datos extraidos como archivo CSV
# se puede aplcair otras opciones para guardar (Base de datos)
df.to_csv('F1Championship.csv')